In [39]:
import pandas as pd
import requests
from pprint import pprint

In [78]:
def get_autor(book):
    string = book['autors']
    index = string.find(']')
    index2 = string[index:].find('[')
    return string[index+1:index2+index]

def get_name(book):
    string = book['name']
    index = string.find(']')
    index2 = string[index:].find('[')
    return string[index+1:index2+index] if index > 0 else string


def get_series_id(book):
    string = book['series']
    if string == '':
        return None
    else:
        index1 = string.find('=')
        index2 = string.find(']')
        return int(string[index1 + 1: index2]) if index1 > 0 else -1


In [113]:
def extract_from_fantlab(series):
    
    url = 'https://api.fantlab.ru/search-editions?q='+str(series)+'&page='

    matches = []
    page = 1
    while True:
        response = requests.get(url+str(page))
        new_matches = response.json()['matches']
        matches += new_matches
        if len(new_matches) == 0:
            break
        page += 1
    
    hits = [m for m in matches if get_series_id(m) == series]

    dic = {'author': [], 'name': [], 'year': []}
    for book in hits:
        dic['author'].append(get_autor(book))
        dic['name'].append(get_name(book))
        dic['year'].append(book['year'])
        
    df = pd.DataFrame(dic)
    df['last_name'] = df['author'].apply(lambda x: x.split(' ')[-1])
    df = df.sort_values(by = ['last_name', 'name'])
    df = df.drop('last_name', axis=1)
    df['year'] = df['year'].astype(int)

    df.to_excel('data/books.xlsx', sheet_name=str(series), header=False, index=False)

In [120]:
#extract_from_fantlab(3186)
extract_from_fantlab(4215)